In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')  # enable import from src/

In [19]:
from pathlib import Path

import numpy as np
import torch

from tqdm import tqdm

from src.dataset import SolutionFeasibilityDataset
from src.net import FeasSatGNN
from src.utils import load_from_wandb

In [6]:
test_data = SolutionFeasibilityDataset(
    [fp for fp in Path('../data/raw/').glob('125_*.json')
        if (int(fp.name.split('_')[1]) >= 20) and
        (int(fp.name.split('_')[2].replace('.json', '')) in np.arange(20,22))],
)
test_data.maybe_initialize()

  0%|          | 0/6 [00:00<?, ?it/s]

wrote problem to file /home/bruno/sat-gnn/notebooks/model.cip
wrote problem to file /home/bruno/sat-gnn/notebooks/model.cip


 17%|█▋        | 1/6 [01:47<08:55, 107.09s/it]

wrote problem to file /home/bruno/sat-gnn/notebooks/model.cip
wrote problem to file /home/bruno/sat-gnn/notebooks/model.cip


 33%|███▎      | 2/6 [03:15<06:24, 96.08s/it] 

...

 67%|██████▋   | 4/6 [06:32<03:16, 98.04s/it]

wrote problem to file /home/bruno/sat-gnn/notebooks/model.cip
wrote problem to file /home/bruno/sat-gnn/notebooks/model.cip


 83%|████████▎ | 5/6 [08:15<01:39, 99.86s/it]

wrote problem to file /home/bruno/sat-gnn/notebooks/model.cip
wrote problem to file /home/bruno/sat-gnn/notebooks/model.cip


100%|██████████| 6/6 [10:05<00:00, 100.85s/it]


In [8]:
net = load_from_wandb(FeasSatGNN(), 'g1xgtzc0', 'sat-gnn')
net

FeasSatGNN(
  (soc_emb): Sequential(
    (0): PreNormLayer()
    (1): Linear(in_features=6, out_features=8, bias=True)
    (2): ReLU()
  )
  (var_emb): Sequential(
    (0): PreNormLayer()
    (1): Linear(in_features=8, out_features=8, bias=True)
    (2): ReLU()
  )
  (con_emb): Sequential(
    (0): PreNormLayer()
    (1): Linear(in_features=4, out_features=8, bias=True)
    (2): ReLU()
  )
  (convs): Sequential(
    (0): HeteroGraphConv(
      (mods): ModuleDict(
        (v2c): GraphConv(in=8, out=8, normalization=both, activation=None)
        (s2c): GraphConv(in=8, out=8, normalization=both, activation=None)
        (c2v): GraphConv(in=8, out=8, normalization=both, activation=None)
        (c2s): GraphConv(in=8, out=8, normalization=both, activation=None)
      )
    )
  )
  (output): Sequential(
    (0): Linear(in_features=8, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=8, bias=True)
    (3): ReLU()
    (4): Linear(in_features=8, out_feature

In [20]:
Y = list()
P_hat = list()
for i in tqdm(list(range(len(test_data)))):
    g, y = test_data[i]

    with torch.no_grad():
        p_hat = torch.sigmoid(net(g)).item()
    
    Y.append(y)
    P_hat.append(p_hat)

  0%|          | 0/6000 [00:00<?, ?it/s]

100%|██████████| 6000/6000 [00:52<00:00, 113.32it/s]


In [23]:
P_hat = np.array(P_hat).reshape(6,1000)
Y = np.array(Y).reshape(6,1000)

In [30]:
Y_hat = (P_hat > 0.5).astype(Y.dtype)
Y_hat

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])

In [38]:
H = (Y_hat == Y).astype(int)

H = {
    20: H[:2],
    22: H[2:4],
    24: H[4:],
}

H_opt = {s: H[s][:,:500] for s in H.keys()}
H_dirty = {s: H[s][:,500:-250] for s in H.keys()}
H_random = {s: H[s][:,-250:] for s in H.keys()}

for size in H.keys():
    print(size)
    print(H[size].sum() / H[size].size)
    print(H_opt[size].sum() / H_opt[size].size)
    print(H_dirty[size].sum() / H_dirty[size].size)
    print(H_random[size].sum() / H_random[size].size)

20
1.0
1.0
1.0
1.0
22
0.875
1.0
0.5
1.0
24
1.0
1.0
1.0
1.0
